# Исследование надежности заемщиков

# Описание проекта

### Контекст

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Задачи

На основе статистики о платёжеспособности клиентов исследовать: влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

### Описание исходных данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### План работы

1. Загрузим данные и изучим общую информацию;
2. Проведем предобработку данных:
    - проверим пропуски;
    - проверим данные на аномалии и исправления;
    - проверим соответствие типов данных;
    - проверим дубликаты;
    - проведем категоризация данных.
3. Ответим на вопросы;
4. Опишем выводы на основе проведенного анализа и дадим рекомендации;

## Обзор данных

In [1]:
# импортируем необходимые библиотеки для проведения анализа

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# загрузим данные для анализа

try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('C:/Users/User/Desktop/Y/Data/Data analyst/3. Предобработка данных/data.csv')

In [3]:
# посмотрим на первые десять строк таблицы

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# посмотрим общую информацию о данных

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Промежуточный вывод 1:**

Судя по первому впечатлению при просмотре информации о таблице, можно сделать следующие наблюдения:

- Есть пропущенные значения в столбцах 'days_employed' и 'total_income';
- Возможно лучше привести значения в столбцах 'days_employed' и 'total_income' к целым числам (из типа 'float64' в 'int64'), чтобы было проще и нагляднее работать с данными;
- В стоблце 'days_employed' есть отрицательные значения, возможно надо взять значения по модулю;
- Проверить уникальные значения каждого столбца для получения полной информации;
- Для корректной группировки и сортировки привести все возможные значения к единой форме, т.е. к единому регистру (например, в столбце 'education' есть значения 'СРЕДНЕЕ', 'Среднее' и 'среднее').

## Предобработка данных

### Заполнение пропусков

In [5]:
# Посмотрим суммарное количество пропусков

df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Возьмем столбец total_income, найдем медианное значение 
# Заполним пропуски медианным значением и посмотрим чему оно равно

total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)
print('Медианное значение по столбцу total_income:', total_income_median)

Медианное значение по столбцу total_income: 145017.93753253992


In [7]:
# Проверим корректность замены

df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

**Промежуточный вывод 2:**

- Чуть более 10% составляют пропущенные значения в столбцах 'days_employed' и 'total_income';
- Возможно это категория безработных либо их данные некорректно были введены;
- В данном случае заполнить пропуски медианным значением лучше, так как некоторые значения в зарплате выделяются и учитывая более 10% пропусков лучше будет заполнить медианным, а не средним значением.

### Проверка данных на аномалии и исправления

In [8]:
# Заменим отрицательные значения дней в столбце 'days_employed'
# Проверим корректность замены

df['days_employed'] = df['days_employed'].abs()
df['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [9]:
# Возьмем столбец 'days-employed', найдем медианное значение 
# Заполним пропуски медианным значением и посмотрим чему оно равно

days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
print('Медианное значение по столбцу days_employed:', days_employed_median)

Медианное значение по столбцу days_employed: 2194.220566878695


In [10]:
# Проверим корректность замены

df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [11]:
# Переведем все значения 'days_employed' пенсионеров из часов в дни
# Оставим все значения в днях, не думаю что они повлияют на выводы задачи

df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df[df['income_type'] == 'пенсионер']['days_employed']/24
df['days_employed']

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14177.753002
             ...     
21520     4529.316663
21521    14330.725172
21522     2113.346888
21523     3112.481705
21524     1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [12]:
# Начнем с первого столбца, проверим уникальные значения

df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [13]:
# Странно, что есть количество детей "-1" и "20"
# Проверим количество строк по количеству детей

df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [14]:
# Возможно ошибка в количестве детей "-1" и "20" при заполнении данных. 
# Думаю, что в данном случае можно заменить "-1" на "1" и "20" на "2"

df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [15]:
# Проверим уникальные значения следующего столбца - возраст клиента

df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

In [16]:
# Есть возраст равный "0". Возможно это ошибка в данных, либо заполняли данные не зная возраст - решили ввести "0"
# Посмотрим количество значений возраста равного "0"

df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Значение возраста "0" упоминается 101 раз.
Есть ли смысл менять значения "0", либо оставить их и они не сильно повлияют на дальнейшую обработку пока непонятно.
Учитывая, что в задании не применяется в выводах возраст клиента, думаю что можно оставить данное значение до конца проекта. 

In [17]:
# Проверим следующий столбец - уровень образования
# Надо будет привести все к единому регистру

df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [18]:
# Следующий столбец - идентификатор уровня образования
# С идентификатором уровня образования все в порядке

df['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [19]:
# Перейдем к семейному положению
# Нужно будет привести все к единому регистру

df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [20]:
# Посмотрим количество идентификаторов семейного положения
# С идентификатором семейного положения тоже все в порядке

df['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [21]:
# Посмотрим столбец пол клиента

df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [22]:
# Посмотрим сколько раз значение "XNA" встречается в столбце

df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Интересно, что есть значение XNA в столбце пола клиента. Посмотрим сколько раз он встречается в столбце
Значение "XNA" встречается лишь раз, думаю пол не повлияет на общие выводы. Оставим значение таким же, так как мы не можем
определить пол ни по каким данным

In [23]:
# Проверим уникальные значения типа занятости

df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [24]:
# Проверим уникальные значения в столбце 'debt'
# Проблем здесь нет, либо имел клиент задолженность по возврату кредитов, либо нет

df['debt'].unique()

array([0, 1], dtype=int64)

In [25]:
# Проверим уникальные значения в столбце ежемесячного дохода
# Данных слишком много, все они разные

df['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

In [26]:
# Проверим уникальные значения в столбце цели получения кредита.
# Есть похожие цели, просто они написаны по разному. Будем проводить категоризацию данных

df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Промежуточный вывод 3:**

- Пропущенные значений были заменены на медианные;
- Заполнил пропуски в столбце 'days_employed' медианным значением;
- При этом произвел замену отрицательных значений на положительные;
- Также, есть значения в которых клиенты работали около 1000 лет и они относятся к категории пенсионеров. Пока непонятно, что делать с такими данными. Скорее всего эти значения указаны в часах, а не в днях.
- Проверил аномальные значения в каждом столбце;
- Заменил отрицательные значения в столбце 'days_employed' на положительные. Заменил значения у пенсионеров с часов на дни;
- Проверил уникальные значения в каждом столбце;
- Произвел замену аномальных значения в столбце 'children' с '-1' на '1' и с '-20' на '2';
- В столбце 'dob_years' нашел аномальное значение равное '0'. Решил его не менять, так как оно не влияет на общую картину задания;
- В столбце 'education_id' необходимо все привести к единому регистру и удалить дубликаты;
- В столбце 'gender' встречается значение XNA, пол определить мы не можем, не обратившись к источнику данных;
- В столбце 'total_income' необходимо все привести к целочисленным значениям;
- В столбце 'purpose' имеются похожие, но написанные по разному цели. Необходимо провести категоризацию данных.

### Изменение типов данных

In [27]:
# Проведем замену с вещественного типа данных на целочисленный в столбцах 'days_employed' и 'total_income'
# Проверим корректность замены

df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Промежуточный вывод 3:**

- Заменили вещественный тип данных на целочисленный в столбцах 'days_employed' и 'total_income';
- Проверили корректность замены, столбцов с вещественным типом данных не осталось.

### Удаление дубликатов

In [28]:
# Приведем значения в столбце 'education_id' к единому регистру и проверим замену

df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [29]:
# Посчитаем общее количество дубликатов в данных

df.duplicated().sum()

71

In [30]:
# 71 дубликат. Возможно были повторно введены данные нескольких клиентов
# Удалим дубликаты и сбросим индекс для дальнейшей работы

df = df.drop_duplicates().reset_index(drop=True)

In [31]:
# Проверим корректность удаления дубликатов

df.duplicated().sum()

0

**Промежуточный вывод 4:**

- Привели все значения в столбце 'education_id' к единому регистру;
- Посчитали общее количество дубликатов. Использовали специальный метод duplicated() и результат передали методу sum(), чтобы вывести общее количество;
- Удалили дубликаты методом drop_duplicates() и сбросили индекс (с изменением новых и удалением старых);
- Дубликаты могли появиться при повторном заполнении заявки клиента в разных точках обслуживания, либо это просто ошибочное создание дубликатов.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [32]:
# Создаем отдельные датафреймы для уровня образования и семейного статуса
# Проверим создание датафреймов

education_dict = df[['education_id', 'education']]
family_dict = df[['family_status_id', 'family_status']]

display(education_dict.head())
display(family_dict.head())

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


In [33]:
# Посчитаем общее количество дубликатов в новом созданном датафрейме

education_dict.duplicated().sum()

21449

In [34]:
# Удалим дубликаты и сбросим индекс

education_dict = education_dict.drop_duplicates().reset_index(drop=True)

In [35]:
# Проверим корректность удаления дубликатов

education_dict.duplicated().sum()

0

In [36]:
# Проведем такую же операцию с другим датафреймом

family_dict.duplicated().sum()

21449

In [37]:
# Удалим дубликаты и сбросим индекс

family_dict = family_dict.drop_duplicates().reset_index(drop=True)

In [38]:
# Проверим корректность удаления дубликатов

family_dict.duplicated().sum()

0

In [39]:
# Удалим из исходного датафрейма столбцы 'education' и 'family_status'
# Проверим корректность удаление столбцов

df = df.drop(columns=['education', 'family_status'])

df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


**Промежуточный вывод 5:**

- Создали два отдельных датафрейма;
- Удалили дубликаты из обоих датафреймов и сбросили индекс
- Удалили столбцы 'education' и 'family_status' из исходного датафрейма.

### Категоризация дохода

In [40]:
# Необходимо провести категоризацию дохода через создание функции с указанием диапазонов зарплат и присвоение им группы
# Создать новый столбец категории зарплат через метод apply()
# Проверим корректность присвоение группы и создания столбца

def total_income_category(total_income):
    if total_income <= 30000:
            return 'E'
    if total_income >= 30001 and total_income < 50000:
            return 'D'
    if total_income >= 50001 and total_income < 200000:
            return 'C'
    if total_income >= 200001 and total_income < 1000000:
            return 'B'
    if total_income >= 1000001:
            return 'A'
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


**Промежуточный вывод 6:**

- Провели категоризацию дохода по диапазонам зарплат в столбце 'total_income' через создание функции;
- Создали новый столбец 'total_income_category' с указанием группы по уровню зарплаты с помощью метода apply(). 

### Категоризация целей кредита

In [41]:
# Необходимо провести категоризацию целей кредита через создание функции с указанием цели по нахождению части значения во всех значениях
# Создать новый столбец целей кредита через метод apply()
# Проверим корректность присвоение цели и создания столбца

def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'

df['purpose_category'] = df['purpose'].apply(purpose_category)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


**Промежуточный вывод 7:**

- Провели категоризацию дохода по целям кредита в столбце 'purpose' через создание функции;
- Создали новый столбец 'purpose_category' с указанием группы по цели кредита с помощью метода apply(). 

## Ответы на вопросы

### Вопрос 1:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [42]:
# Для проверки зависимости вызываем метод pivot_table() для подготовки сводных таблиц
# Посчитаем зависимость и выведем результат в процентах
# Отчего-то в последней строке появилось значение NaN, хотя я заполнил все пропуски
# На выводах это не сказывается

children_status_result = df.pivot_table(index = 'children', columns='debt', values='days_employed', aggfunc='count')
children_status_result ['result_children_status'] = children_status_result[1] / children_status_result[0] * 100
display(children_status_result)

debt,0,1,result_children_status
children,,,
0,13028.0,1063.0,8.159349
1,4410.0,445.0,10.090703
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


**Вывод 1:**

**По полученным данным видно, что на процент возврата кредита в срок кардинально не влияет количество детей клиента.**

### Вопрос 2:

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [43]:
# Для проверки зависимости вызываем метод pivot_table() для подготовки сводных таблиц
# Посчитаем зависимость и выведем результат в процентах
# Добавим столбец из словаря для расшифровки семейного статуса

family_status_result = df.pivot_table(index='family_status_id', columns= 'debt', values='days_employed', aggfunc='count')
family_status_result ['result_family_status'] = family_status_result[1] / family_status_result[0] * 100
family_status_result.reset_index().merge(family_dict)

,family_status_id,0,1,result_family_status,family_status
0,0,11408,931,8.160940,женат / замужем
1,1,3763,388,10.310922,гражданский брак
2,2,896,63,7.031250,вдовец / вдова
3,3,1110,85,7.657658,в разводе
4,4,2536,274,10.804416,Не женат / не замужем


**Вывод 2:**

**Семейное положение влияет на возврат кредита в срок. Самые низкие показатели, на удивление - у вдовцов/вдов и тех, кто в разводе. Люди в гражданском браке и не будучи в браке допускают большую вероятность невозврата кредита в срок**

### Вопрос 3:

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [44]:
# Для проверки зависимости вызываем метод pivot_table() для подготовки сводных таблиц
# Посчитаем зависимость и выведем результат в процентах

total_income_result = df.pivot_table(index = 'total_income_category', columns='debt', values='days_employed', aggfunc='count')
total_income_result ['result_total_income'] = total_income_result[1] / total_income_result[0] * 100
display(total_income_result)

debt,0,1,result_total_income
total_income_category,,,
A,23,2,8.695652
B,4685,356,7.598719
C,14656,1360,9.279476
D,329,21,6.382979
E,20,2,10.000000


**Вывод 3:**

**Зависимость между уровнем дохода и возвратом кредита в срок незначительна. Больше всего кредитов выдают людям со средним  и выше среднего уровнем дохода. Учитывая разницу в количестве выданных кредитов, значения возвратов кредитов в срок примерно одинаковы.**

### Вопрос 4:

**Как разные цели кредита влияют на его возврат в срок?**

In [45]:
# Для проверки зависимости вызываем метод pivot_table() для подготовки сводных таблиц
# Посчитаем зависимость и выведем результат в процентах

purpose_result = df.pivot_table(index = 'purpose_category', columns='debt', values='days_employed', aggfunc='count')
purpose_result ['result_total_income'] = purpose_result[1] / purpose_result[0] * 100
display(purpose_result)

debt,0,1,result_total_income
purpose_category,,,
операции с автомобилем,3903,403,10.325391
операции с недвижимостью,10029,782,7.797388
получение образования,3643,370,10.156464
проведение свадьбы,2138,186,8.699719


**Вывод 4:**

**Цели кредита влияют на его возврат в срок. На удивление, сложнее всего возвращают в срок клиенты, которые брали кредиты для операций с автомобилем, а также для получения образования, которые чаще всего не сразу окупаются**

## Общий вывод:

На основании статистики о платёжеспособности клиентов банка была проделана следующая работа:

**1. Проведена предобработка данных со следующими изменениями:**

   - Заполнили пропущенные значения в столбцах "Общего трудового стажа в днях" и "Ежемесячного дохода". Заменили пустые ячейки на медианные значения, равные *2194* и *145017* соответственно;
   - Провели изменения типов данных. Все возможные значения привели из дробных в целые числа для удобства просмотра данных;
   - Провели проверку уникальных значений во всех столбцах. Произвели замену в столбце "Количества детей в семье" с "-1" на "1" и с "20" на "2", потому что это смотрелось самым логичным выводом;
   - Провели удаление *71* дубликата и привели все названия к единому регистру;
   - Провели категоризацию дохода по диапазонам ежемесячного дохода и категоризацию по целям получения кредита с созданием соответствующих групп.
    
    
**2. Согласно поставленной задаче, отвечаем на целевые вопросы:**
    
   1) Есть ли зависимость между количеством детей и возвратом кредита в срок?
    
   `На процент возврата кредита в срок кардинально не влияет количество детей клиента`, однако все же самые низкий процент невозврата кредита в срок равный *8,15%* у клиентов без детей, при том что наибольшее количество кредитов (*более 13000*) было выдано именно данной категории клиентов. У клиентов процент невозврата кредита в срок близок к *9%* с 3 детьми и более *10%* с 1, 2 и 4 детьми.
    
   2) Есть ли зависимость между семейным положением и возвратом кредита в срок?
    
   `Семейное положение имеет влияние на возврат кредита в срок`. Самые низкие показатели, на удивление - у вдовцов/вдов (*около 7%*) и тех, кто в разводе (*около 7,5%*). Люди в гражданском браке и не будучи в браке допускают большую вероятность невозврата кредита в срок (*более 10%*), однако и количество выданных кредитов у этих категорий больше. У официально зарегестрировавших семейные отношения клиентов процент невозврата средний (*около 8%*), при этом было одобрено более *10000* кредитов.
    
   3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    
   `Зависимость между уровнем дохода и возвратом кредита в срок незначительна`, если учитывать разницу в количестве выданных кредитов по категориям дохода. Больше всего кредитов выдают людям со средним уровнем дохода (*от 50 до 200 тысяч*) с процентом невозврата кредита в срок *более 9%* и выше среднего уровнем дохода (*от 200 тысяч до 1 миллиона*) с процентом невозврата *около 7,5%*. Самый лучший показатель возврата кредитов в срок (*около 6,4%*) у клиентов с доходом *от 30 до 50 тысяч*.
    
   4) Как разные цели кредита влияют на его возврат в срок?
    
   `Цели кредита влияют на его возврат в срок`. На удивление, сложнее всего возвращают в срок клиенты, которые брали кредиты для операций с автомобилем и получения образования, которые чаще всего не сразу окупаются или не окупаются вовсе, и процент невозврата кредитов в срок данных категорий составляет *более 10%*. Более низкий процент невозврата в срок имеют клиенты, получающие кредиты на операции с недвижимостью (*около 7,8%*) и проведения свадеб (*около 8,7%*).
    
   На основании вышеуказанных промежуточных выводов можно сформировать модели по способности потенциального клиента вовремя вернуть кредит банку:
    
   - **Добросовестный клиент** - без детей, находится или был в браке (официальном), с уровнем дохода от 30  до 50 тысяч, запрашивающий кредит на операции с недвижимостью;
    
   - **Среднестатистический клиент** - без детей, находится или был в браке (официальном), с уровнем дохода от 30 тысяч до 1 миллиона, запрашивающий кредит на операции с недвижимостью или проведение свадьбы;
    
   - **Недобросовестный клиент** - имеет от 1 ребенка, не в отношениях или гражданском браке, с уровнем дохода менее 30 тысяч и более 1 миллиона, запрашивающий кредит на операции с автомобилем или получения образования. 
    
    
**3. По итогам проведенной работы можно рассмотреть рекомендации для модели кредитного скоринга банка по улучшению качества входных данных:**

   - Присвоение каждому клиенту идентификационного номера во избежании дублирования данных;
   - Заполнение всех данных по клиенту, ведь от этого будет зависеть анализ даннных и решение по его запросу на кредит;
   - Заполнение стажа работы в годах, так легче будет получить точную информацию от клиента;
   - Во избежание заполнения аномальных и некорректных значений в системе можно установить запрет на указание отрицательных значений в столбцах с количеством детей, стажем работы, ежемесячного дохода.